In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet



In [2]:
df= pd.read_csv('Retail 2009-10.csv')
print(df.head())
print(df.info())
print(df.describe())
print(df.shape)

#CSV INFO
# Rows - 525461
# Columns - 8
# Columns - InvoiceNo[0], StockCode[1], Description[2], Quantity[3], InvoiceDate[4], UnitPrice[5], CustomerID[6], Country[7]
# Price = UnitPrice * Quantity


  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

        InvoiceDate  Price  Customer ID         Country  
0  01-12-2009 07:45   6.95      13085.0  United Kingdom  
1  01-12-2009 07:45   6.75      13085.0  United Kingdom  
2  01-12-2009 07:45   6.75      13085.0  United Kingdom  
3  01-12-2009 07:45   2.10      13085.0  United Kingdom  
4  01-12-2009 07:45   1.25      13085.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  

In [ ]:
#Rows with non-positive unit prices were removed as invalid transactions. Returns, represented by negative quantities, may or may not appear depending on the underlying data distribution.
df = df.rename(columns={'Price': 'UnitPrice'})
df['TotalPrice'] = df['UnitPrice'] * df['Quantity']
print(len(df))
df = df.drop_duplicates()
print(len(df))
df = df[df['UnitPrice'] >=0]# remove non-positive unit prices
df = df.dropna(subset=['InvoiceDate'])
print(len(df))
print(df.head())

514911
514911
514911
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

  InvoiceDate  UnitPrice  Customer ID         Country  TotalPrice  
0  2009-12-01       6.95      13085.0  United Kingdom        83.4  
1  2009-12-01       6.75      13085.0  United Kingdom        81.0  
2  2009-12-01       6.75      13085.0  United Kingdom        81.0  
3  2009-12-01       2.10      13085.0  United Kingdom       100.8  
4  2009-12-01       1.25      13085.0  United Kingdom        30.0  


In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='raise')
# convert to monthly period then back to timestamps (month start) to avoid non-fixed frequency error
df['InvoiceDate'] = df['InvoiceDate'].dt.to_period('M').dt.to_timestamp()
df.head()



,Invoice,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,30.0


In [ ]:
from itertools import count


print(count(df['TotalPrice']<0))# check for negative total prices 

count(0         False
1         False
2         False
3         False
4         False
          ...  
525456    False
525457    False
525458    False
525459    False
525460    False
Name: TotalPrice, Length: 514911, dtype: bool)


In [22]:
#Number of unique countries 
print(df['Country'].nunique())
#Identify top 3 by transaction count or sales
print(df['Country'].value_counts().head(3))

40
Country
United Kingdom    475372
EIRE                9659
Germany             8113
Name: count, dtype: int64


In [ ]:
#groupby country and month
country_monthly = df.groupby(['Country', 'InvoiceDate']).agg({'TotalPrice': 'sum'}).reset_index()
country_monthly.head()
#aggregate total sales by month
monthly_sales = df.groupby('InvoiceDate').agg({'TotalPrice': 'sum'}).reset_index()
monthly_sales = monthly_sales.rename(columns={'InvoiceDate': 'ds', 'TotalPrice': 'y'})
monthly_sales.head()

,Country,InvoiceDate,TotalPrice
0,Australia,2009-12-01,50.25
1,Australia,2010-02-01,1029.66
2,Australia,2010-03-01,429.39
3,Australia,2010-04-01,630.95
4,Australia,2010-05-01,1312.70
